In [1]:
import pandas as pd
from wrappers import *
import os
from bs4 import BeautifulSoup
import zipfile
import shutil
import requests
import json

In [2]:
"""
Demo for central library scraping
"""
from updated_scraper import *
s = PkuScraper()

# Important:
# After you log in with Chrome, with VPN on and pkulaw signed in, right click anywhere on the page and direct to "Inspect", find "Network", zoom into one event, and paste the Cookie here
s.set_input_type('keyword')
s.set_cookies('pkulaw_v6_sessionid=qm4iqggkoktau3szdek4bfie; Hm_lvt_8266968662c086f34b2a3e2ae9014bf8=1630066629,1631060463; authormes=5f8e703755203640a7baa1c56e09ce5616c7268dca732f3b39028526dff6560a10f8eedcbd05b937bdfb; xCloseNew=25; TKMessage=YAfPC0z71+dAP6IuGWuNiRH0v+qdVzh+B1gzi7v+4HpzrjkqxpCE182DCs3h6rArIK5FdodPwnXrv3F4oUTIaxdAVfE5YNjdtOkOMEKd+Zi07VY7nN1ZJ1+/4UODddYR3bG+72Wm7sXKd5oOAjquTO5ZJpx01f8Q2jJRANmZ0+AJVtLDktc17eMfPKswBebq4EKW82+y0EmTGAMQa8fTLI3/q7euD4QTBsC1wAc6oR0tnuaXg+5iMMiz49VBXFQ2; Hm_lpvt_8266968662c086f34b2a3e2ae9014bf8=1632582141')
s.library = 'lar' # lar: library for local govt documents. chl: library for central documents
s.set_filters(lib_='lar', type_=['unspecified'], dept_=['unspecified'], effective_=['unspecified']) # For other choices please refer to the code


In [3]:
s.edit_postform({'Aggs.Category': '001', 'library': 'lar', 'Aggs.IssueDate': '2021'})
soup = s.search_page_V6('', _debug=True)
int(soup.find("span" ,class_="total").find('strong').text)

269

In [4]:
# First, we would need to get a list of id's for more advanced searching
temp = s.get_locality_id()

jsonString = temp.text.replace('\r', '').replace('\n', '')
dict_list = json.loads(jsonString)

pd.DataFrame(dict_list).to_excel('/Users/kevin/Dropbox/Pkulaw_scraper/dept_id_name.xlsx', index=False)

In [4]:
df = pd.read_excel('/Users/kevin/Dropbox/Pkulaw_scraper/dept_id_name.xlsx')

def MyInt(x):
    try:
        return int(x)
    except:
        return -8888

df.id = df.id.apply(lambda x: MyInt(x))
df = df[(df.id > 1e4) & (df.id < 1e5)]

In [6]:
# We're going to do the searches by prefecture / year, and collect all its corresponding category info
for year in range(1990, 2020):
    for id_ in df.id.unique():
        temp = r'{"RelatedPrompted":"","EffectivenessDic":"","SpecialType":"","IssueDepartment":'+str(id_)+r',"TimelinessDic":"","Category":"","IssueDate":"'+str(year)+'"}'
        s.edit_postform({'Aggs': temp})
        temp = s.get_category_dist(_debug=True)
        jsonString = temp.text.replace(r'\r', '').replace(r'\n', '')
        dict_list = json.loads(jsonString)
        pd.DataFrame(dict_list).to_excel('/Users/kevin/Dropbox/Pkulaw_scraper/data/'+str(id_)+'_'+str(year)+'.xlsx', index=False)
        time.sleep(1)

In [ ]:

# One first thing we can do is to simulate the searching process:
print(s.search_page_V6('证照分离'))
# It will return a list where one of the sublists document the titles, one document the urls, and the other the id's

# Once you get a url, we can get the full text of the paper from this function call
print(s.decode_page_v6(r"https://www.pkulaw.com/chl/7e71a76b28fb96efbdfb.html?keyword=%E8%AF%95%E7%82%B9"))
# It returns a Soup object containing the texts of documents

# If we don't want to visit the website repeatedly for those documents, we can download them once and for all.
s.set_ouput_directory(r'/Users/kevin/Desktop/temp.zip')
s.download('f3c7b1f7db0795fcbdfb,69425ff77e3fd798bdfb,a8decc3fc18fb591bdfb,2c8dfe7452186879bdfb,78c556b1def5fb57bdfb,87899185ab4ffaadbdfb')

In [8]:
# You might also want to unzip the documents you downloaded.
def un_zip(file_name):
    # unzip zip file
    if not file_name.endswith('.zip'):
        return
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        try:
            zip_file.extract(names, file_name + r"_files/")
        except:
            pass
    zip_file.close()

In [30]:

raise ValueError
# ===========================================================
# DO NOT RUN: Some sample code for reference (how to unzip)
# Init Dirs
src_path = '/Users/kevin/Desktop/tmp.zip_files/'
dest_path = '/Users/kevin/Dropbox/Policy Experimentation/Data/Raw Webpages/Local/'
zip_path = '/Users/kevin/Desktop/tmp.zip'

counter = 0
maxlen = df_local.shape[0]
while counter < maxlen:
    download(','.join(df_local['gid'][counter:counter+10]))
    un_zip(zip_path)
    file_list = os.listdir(src_path)
    if len(file_list) > 0:
        for file in file_list:
            counter += 1
            #time.sleep(1)
            if counter % 100 == 0:
                print('Progress:', counter)
                time.sleep(20)
            try:
                # unzip and move directory
                os.rename(src_path+file, src_path + str(counter) + '.html')
                shutil.move(src_path + str(counter) + '.html', dest_path + str(counter) + '.html')
            except:
                pass
# ===========================================================    

BadZipFile: File is not a zip file